# DAG full_LE

In [1]:
import re
from typing import Any, Dict, List
import pymysql
# from airflow import DAG
# from airflow.operators.python import PythonOperator
import os 
import datetime
import requests
import time
from sqlalchemy import create_engine
import numpy as np
import openpyxl
import gspread
import gspread_dataframe as gd
from tqdm import tqdm
from sys import exit
import pandas as pd
import sys
import glob
link_to_table = 'https://docs.google.com/spreadsheets/d/1A1TyZPeq9r2HiDi-CHfmTfRpuAQYToib1vqyTx4ZyRE/edit#gid=0'
df = pd.read_excel('/'.join(link_to_table.split('/')[:-1])+'/export?format=xlsx')
PATH, df = [dict(df.loc[x]) for x in list(df.index)], None
i = 0
while len([py for py in glob.glob(f"{PATH[i]['path_downloads']}*.xlsx")]) == 0:
    i +=1
path_downloads = PATH[i]['path_downloads']
path_algoritm = PATH[i]['path_algoritm']
sys.path.append(f'{path_algoritm}Модули')
import constants_for_marketplace_metrics_dags as const
import working_with_marketplace_metrics_dags as wwm

bd_param = const.bd_param
engine_param = const.engine_param
maslow_json = path_downloads + const.maslow_json
gc = gspread.service_account(filename=maslow_json)
args = const.args
os.environ['NO_PROXY'] = 'URL'

def le_orders_metrics(path_le_orders_metrics):
    df = pd.read_excel(wwm.path_full_LE_df,
                       sheet_name='le_orders_metrics',
                       dtype={'Артикул': 'str'})
    df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
    df = wwm.clear_article(df, 'Артикул')
    df = df.fillna(0)
    df = df.groupby('Артикул').sum().reset_index(drop=False)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
    if len(wwm.date_res_table(df)) != len(df.columns[1:]):
        exit()
    wwm.unload_metrics(path_metrics=path_le_orders_metrics, 
                       data1=df, 
                       column='Артикул')
    T_le_orders_metrics = wwm.get_T_metrics(path_metrics=path_le_orders_metrics)
    sh = gc.open("full_LE")
    list_name = 'le_orders_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_le_orders_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True) 
    
def le_fbo_stocks_metrics(path_le_fbo_stocks_metrics):
    df = pd.read_excel(wwm.path_full_LE_df,
                       sheet_name='stocks_le_metrics',
                       dtype={'Артикул': 'str'})
    df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
    df = wwm.clear_article(df, 'Артикул')
    df = df.fillna(0)
    df = df.groupby('Артикул').sum().reset_index(drop=False)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
    if len(wwm.date_res_table(df)) != len(df.columns[1:]):
        exit()
    wwm.unload_metrics(path_metrics=path_le_fbo_stocks_metrics, 
                       data1=df, 
                       column='Артикул')
    T_le_fbo_stocks_metrics = wwm.get_T_metrics(path_metrics=path_le_fbo_stocks_metrics)
    sh = gc.open("full_LE")
    list_name = 'le_fbo_stocks_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_le_fbo_stocks_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True) 

def le_price_metrics(path_le_price_metrics):
    df = pd.read_excel(wwm.path_full_LE_df,
                       sheet_name='le_price_metrics',
                       dtype={'Артикул': 'str'})
    df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
    for i in tqdm(range(len(df))):
        price_not_null = 0
        for col in df.columns[1:]:
            if str(df.loc[i][col]) != 'nan':
                price_not_null = df.loc[i][col]
            else:
                df.loc[i, col] = price_not_null
    df = wwm.clear_article(df, 'Артикул')
    df = df.fillna(0)
    df = df.groupby('Артикул').mean().reset_index(drop=False)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
    if len(wwm.date_res_table(df)) != len(df.columns[1:]):
        exit()
    wwm.unload_metrics(path_metrics=path_le_price_metrics, 
                       data1=df, 
                       column='Артикул')
    T_le_price_metrics = wwm.get_T_metrics(path_metrics=path_le_price_metrics)
    sh = gc.open("full_LE")
    list_name = 'le_price_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_le_price_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True)
    
# with DAG(
#     dag_id='full_LE_dag',
#     default_args=args,
#     schedule_interval='00 06 * * *',
#     start_date=datetime.datetime(2023, 11, 13),
#     catchup=False,
#     dagrun_timeout=datetime.timedelta(minutes=10)
# ) as dag:

#     le_orders_metrics_task = PythonOperator(task_id = 'le_orders_metrics',
#                                             python_callable=le_orders_metrics, 
#                                             op_kwargs={'path_le_orders_metrics': wwm.path_le_orders_metrics},
#                                             dag=dag)
    
#     le_fbo_stocks_metrics_task = PythonOperator(task_id = 'le_fbo_stocks_metrics',
#                                           python_callable=le_fbo_stocks_metrics, 
#                                           op_kwargs={'path_le_fbo_stocks_metrics': wwm.path_le_fbo_stocks_metrics},
#                                           dag=dag)
    
#     le_price_metrics_task = PythonOperator(task_id = 'le_price_metrics',
#                                            python_callable=le_price_metrics, 
#                                            op_kwargs={'path_le_price_metrics': wwm.path_le_price_metrics},
#                                            dag=dag)
    
#     le_orders_metrics_task >> \
#     le_fbo_stocks_metrics_task >> \
#     le_price_metrics_task

In [2]:
le_orders_metrics(wwm.path_le_orders_metrics)

Были изменения, датафрейм выгружен


In [3]:
le_fbo_stocks_metrics(wwm.path_le_fbo_stocks_metrics)

Были изменения, датафрейм выгружен


In [4]:
le_price_metrics(wwm.path_le_price_metrics)

100%|████████████████████████████████████████| 169/169 [00:00<00:00, 503.92it/s]


Были изменения, датафрейм выгружен


In [ ]:
    df = pd.read_excel(wwm.path_full_LE_df,
                       sheet_name='le_orders_metrics',
                       dtype={'Артикул': 'str'})
    df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
    df = wwm.clear_article(df, 'Артикул')
    df = df.fillna(0)
    df = df.groupby('Артикул').sum().reset_index(drop=False)
    df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
    if len(wwm.date_res_table(df)) != len(df.columns[1:]):
        exit()
    wwm.unload_metrics(path_metrics=path_le_orders_metrics, 
                       data1=df, 
                       column='Артикул')
    T_le_orders_metrics = wwm.get_T_metrics(path_metrics=path_le_orders_metrics)
    sh = gc.open("full_LE")
    list_name = 'le_orders_metrics'
    i = [x for x in list(range(len(sh.worksheets()))) if list_name in str(sh.worksheets()[x])][0]
    ws = sh.get_worksheet(i)
    ws.clear()
    gd.set_with_dataframe(worksheet=ws,
                          dataframe=T_le_orders_metrics, 
                          include_index=True,
                          include_column_header=True,
                          resize=True) 

In [2]:
df = pd.read_excel(wwm.path_full_LE_df,
                   sheet_name='le_orders_metrics',
                   dtype={'Артикул': 'str'})
df = wwm.data_columns_sorted(df[['Артикул']+wwm.dt_col(df)])[['Артикул']+list(df.columns[-10:])]
df = wwm.clear_article(df, 'Артикул')
df = df.fillna(0)
df = df.groupby('Артикул').sum().reset_index(drop=False)
df.iloc[:, 1:] = df.iloc[:, 1:].astype(int)
if len(wwm.date_res_table(df)) != len(df.columns[1:]):
    exit()

SystemExit: 

/Users/dmitriymaslov/anaconda3/envs/work_venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3534 UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.

In [3]:
df

,Артикул,09-February-2024,10-February-2024,11-February-2024,12-February-2024,14-February-2024,15-February-2024,16-February-2024,17-February-2024,18-February-2024,19-February-2024
0,19200,1,0,0,0,0,2,0,0,0,0
1,19201,0,0,1,0,0,0,0,0,1,0
2,19202,0,0,1,3,2,5,0,1,0,0
3,19203,2,0,0,0,1,1,0,0,0,0
4,19204,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
175,J0505,0,0,1,0,0,0,0,0,0,0
176,J0561,0,0,0,0,0,2,2,2,1,0
177,K0001,0,0,0,1,0,0,0,1,0,0
178,K0052,0,0,0,0,0,0,0,0,0,0
